## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
bfrom selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException


def start_browser(headless = True):
    if headless == True:
        option = webdriver.ChromeOptions()
        option.add_argument('--headless')
        browser = webdriver.Chrome(options = option)
    else:
        browser = webdriver.Chrome()

    browser.get('https://www.winespectator.com/auth/login')

    username = browser.find_element_by_name("userid")
    username.clear()
    username.send_keys('johnmkline@gmail.com')

    password = browser.find_element_by_name("passwd")
    password.clear()
    password.send_keys('ohnomoose')

    browser.find_element_by_id("target").click()

def get_text(sixdigstring):
    browser.get(f'https://www.winespectator.com/wine/detail/source/search/note_id/{sixdigstring}')
    full_site = browser.find_element_by_class_name("main-layout")
    text_string = full_site.text
    text_string = text_string[:text_string.find('Add this wine to this Personal Wine List')]
    return text_string

def iterate_pages(index_list):
    index0_num = int(index_list[0])
    list_len = len(index_list)
    with open(f'raw_reviews_i{index0_num}_l{list_len}.txt','w') as file:
        file.write("{")
        for string in index_list:
            file.write(string+' : '+get_text(string)+", ")
        file.write("'end': None}")


In [1]:
from itertools import islice


def clean_file(filename):
    # Open file and strip
    with open(filename) as tempfile:
        full_file = tempfile.readlines()
    full_file = list(map(lambda x: x.strip(), full_file))
    
    # Check / clean first line
    if full_file[0][0] == '{':
        full_file[0] = full_file[0][1:]

    # Assign webpage content to page_dict
    current_page = None
    current_linelist = []
    for line in full_file:
        if line.find(':') >=0:
            test = line[0:line.find(':')].strip(',').strip()
            if test.isdigit():
                if current_page != None:
                    consol_page_dict[current_page] = current_linelist
                current_linelist = []
                
                current_page = line[0:line.find(':')]
                current_page = current_page.strip(',').strip()
                current_linelist.append(line[line.find(':')+1:])
            else:
                current_linelist.append(line)
        else:
            current_linelist.append(line)




In [9]:
def save_object(object,filename):
    with open(filename, 'w') as f:
        json.dump(object, f)

def load_object(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return(data)    

In [3]:
filelist = []
for i in range(6):
    filelist.append(f'raw_reviews_{i}.txt')
print(filelist)

['raw_reviews_0.txt', 'raw_reviews_1.txt', 'raw_reviews_2.txt', 'raw_reviews_3.txt', 'raw_reviews_4.txt', 'raw_reviews_5.txt']


In [90]:
from datetime import datetime
import json


consol_page_dict = load_object('consol_pages_2020-02-13-16-12.json')

for item in filelist:
    clean_file(item)

save_object(consol_page_dict,'consol_pages_'+datetime.now().strftime("%Y-%m-%d-%H-%M")+'.json')

# Updating index remaining tracker
keylist = list(consol_page_dict.keys())
all_index = []
number = 999999
for i in range(number):
    all_index.append(str(i+1).zfill(6))
remaining_index = all_index
for item in keylist:
    remaining_index.remove(item)
save_object(remaining_index,'remaining_index.json')

print('Keys Completed: ',len(keylist))
print('Keys Remaining: ',len(remaining_index))
print('You are ', round(100*len(keylist)/999999,2),' "%" of the way there!')

Keys Completed:  140513
Keys Remaining:  859486
You are  14.05  "%" of the way there!


In [7]:
print(remaining_index[0:100])


['036488', '036489', '036490', '036491', '036492', '036493', '036494', '036495', '036496', '036497', '036498', '036499', '036500', '036501', '036502', '036503', '036504', '036505', '036506', '036507', '036508', '036509', '036510', '036511', '036512', '036513', '036514', '036515', '036516', '036517', '036518', '036519', '036520', '036521', '036522', '036523', '036524', '036525', '036526', '036527', '036528', '036529', '036530', '036531', '036532', '036533', '036534', '036535', '036536', '036537', '036538', '036539', '036540', '036541', '036542', '036543', '036544', '036545', '036546', '036547', '036548', '036549', '036550', '036551', '036552', '036553', '036554', '036555', '036556', '036557', '036558', '036559', '036560', '036561', '036562', '036563', '036564', '036565', '036566', '036567', '036568', '036569', '036570', '036571', '036572', '036573', '036574', '036575', '036576', '036577', '036578', '036579', '036580', '036581', '036582', '036583', '036584', '036585', '036586', '036587']

In [91]:
#full_dict = load_object('consol_pages_2020-02-13-11-35.json')

#test_entry = full_dict['044440']

#print(type(full_dict))

corkcount = 0
corkkeys = []

for item in keylist:
    if full_dict[item][0] == ' This page is corked.':
        corkcount+=1
        corkkeys.append(item)

print(corkcount)
print(corkkeys)

corkkeys.sort()

KeyError: '036498'

In [82]:
def cork_counter(corkkey):
    text_list = test_dict[corkkey]
    corkcount = 0    
    for item in text_list:
        if ' This page is corked.' in item:
            corkcount += 1
    return corkcount

cork_text = [' This page is corked.']

def cork_splitter(corkkey):
    text_list = test_dict[corkkey]
    count = cork_counter(corkkey)

    results = {}
    results['keylist'] = []
    results['textlist'] = []

    for i in range(count-1):
        key = text_list[1+i*2+1][12:18]
        results['keylist'].append(key)
        results['textlist'].append(cork_text)

    results['keylist'].append(text_list[1+(count-1)*2+1][12:18])
    realtext  = []
    for i in range(3):
        realtext.append(None)
    for item in text_list[1+(count-1)*2+4:]:
        realtext.append(item)
    results['textlist'].append(realtext)
    return results

print(cork_splitter(corkkeys[100]))

{'keylist': ['095047', '095048', '095049', '095050', '095051', '095052', '095053', '095054'], 'textlist': [[' This page is corked.'], [' This page is corked.'], [' This page is corked.'], [' This page is corked.'], [' This page is corked.'], [' This page is corked.'], [' This page is corked.'], [None, None, None, 'JOSEPH DROUHIN', 'Chablis Sécher 1999', 'Score: 84', 'Release Price $30', 'Country France', 'Region Burgundy', 'Issue May 31, 2001', 'Your Rating', 'You have not yet rated this wine.', 'Tasting Note', 'Some nice ripe fruit shows up in this medium-bodied Chablis--tropical, pear, lemon. Fresh and clean, with a slight mineral edge on the crisp finish. Drink now. — PM']]}


In [87]:
test_dict = load_object('consol_pages_2020-01-29-10-17.json')
test_keylist = list(test_dict.keys())
test_count = 0
test_corkkeys = []

for item in test_keylist:
    if test_dict[item][0] == ' This page is corked.':
        test_count+=1
        test_corkkeys.append(item)


for item in test_corkkeys:
    print(test_dict[item])


print(test_count)
print(test_corkkeys)






, 'Médoc 1987', 'Score: 80', 'Release Price $11', 'Country France', 'Region Bordeaux', 'Issue Nov 30, 1989', 'Your Rating', 'You have not yet rated this wine.', 'Tasting Note', '— JS']
[' This page is corked.', 'The page you have requested does not seem to exist.', 'Return Hom, 013697 : — Ratings —', 'Tasting Note', 'Ratings Ratings Search Daily Picks Tasting Reports Insider Weekly Advance Values Vintage Charts About Our Tastings', 'CHÂTEAU DE LAMARQUE', 'Haut-Médoc 1987', 'Score: 74', 'Release Price $10', 'Country France', 'Region Bordeaux', 'Issue Nov 30, 1989', 'Your Rating', 'You have not yet rated this wine.', 'Tasting Note', '— JS']
[' This page is corked.', 'The page you have requested does not seem to exist.', 'Return Hom, 013724 : — Ratings —', 'Tasting Note', 'Ratings Ratings Search Daily Picks Tasting Reports Insider Weekly Advance Values Vintage Charts About Our Tastings', 'DROUHIN-LAROZE', 'Bonnes Mares 1987', 'Score: 89', 'Release Price $38', 'Country France', 'Region Bur

In [89]:
#print(test_dict[test_corkkeys[0]])
for item in test_corkkeys[0]:
    corkdict = cork_splitter(item)
    if len(corkdict['keylist'])==1:
        test_dict[corkdict['keylist'][0]] = corkdict['textlist'][0]
        #put activity here
    else:
        for i in range(len(corkdict['keylist'])-1):
            test_dict[corkdict['keylist'][i]] = corkdict['textlist'][i]
        test_dict[corkdict['keylist'][-1]] = corkdict['textlist'][-1]
    test_dict[item] = cork_text

KeyError: '0'

In [36]:
test_entry = full_dict['000001']
print(test_entry)

[' — Ratings —', 'Tasting Note', 'Ratings Ratings Search Daily Picks Tasting Reports Insider Weekly Advance Values Vintage Charts About Our Tastings', 'JACQUESSON', 'Brut Champagne Degorge Tardive 1975', 'Score: 90', 'Release Price $NA', 'Country France', 'Region Champagne', 'Issue Dec 31, 1993', 'Your Rating', 'You have not yet rated this wine.', 'Tasting Note', 'A classical example of a very mature Champagne. An old, gentle wine with fine bubbles and smoky, apple, sherry, yeasty, and toasted oak aromas and flavors. Very good acidity still. Drink now. —']


In [23]:
def clean_basecase(linelist):
    # Select for useful rows
    relevant = linelist[3:10]
    relevant.append(linelist[13])

    # Convert to dict
    temp_dict = {}

    #Vineyard
    vineyard = relevant[0].title()
    temp_dict['vineyard'] = vineyard

    #Name
    name = relevant[1].title()[0:-5]
    temp_dict['name'] = name

    #Year
    year = relevant[1][-4:]
    temp_dict['year'] = year

    #Score
    score = relevant[2][-3:]
    temp_dict['score'] = score

    #Release Price
    price = relevant[3][15:]
    if price == "NA":
        price = None
    temp_dict['price'] = price

    #Country
    country = relevant[4].title()[8:]
    temp_dict['country'] = country

    #Region
    region = relevant[5].title()[7:]
    temp_dict['region'] = region

    #Review Issue
    review_date = relevant[6]
    temp_dict['review_date'] = review_date

    #Tasting Note
    taste_note = relevant[7][:-2].lower()
    temp_dict['taste_note'] = taste_note

    return(temp_dict)

print(clean_basecase(test_entry))

{'vineyard': 'Jacquesson', 'name': 'Brut Champagne Degorge Tardive', 'year': '1975', 'score': ' 90', 'price': None, 'country': 'France', 'region': 'Champagne', 'review_date': 'Issue Dec 31, 1993', 'taste_note': 'a classical example of a very mature champagne. an old, gentle wine with fine bubbles and smoky, apple, sherry, yeasty, and toasted oak aromas and flavors. very good acidity still. drink now.'}
